In [ ]:
import boto3
import pandas as pd
import numpy as np
from pathlib import Path
import json
import re

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

from sklearn.ensemble import forest, gradient_boosting
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_selection import SelectFromModel, SelectKBest

import mlflow
from mlflow import log_metric, log_param, log_artifact
from mlflow.sklearn import log_model
from sklearn.model_selection import ParameterGrid
#import shap

In [ ]:
shap.initjs()

In [ ]:
train_start_date = '2017-01-01'
train_end_date = '2019-03-01'

In [ ]:
data_path = Path('/code/data/raw')
data_path.mkdir(parents=True, exist_ok=True)

# Get Avante Data

In [ ]:
secret_name = "cust_db_credentials"
region_name = "us-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
client = session.client(service_name="secretsmanager", region_name=region_name)

get_secret_value_response = client.get_secret_value(SecretId=secret_name)

db_info = json.loads(get_secret_value_response["SecretString"])

avan_connect_url = URL(
    drivername="mssql+pyodbc",
    username=db_info["username"],
    password=db_info["password"],
    host=db_info["host"],
    database="AVAN",
    query={'driver': 'ODBC Driver 17 for SQL Server'}
)
'D1'
avan_engine = create_engine(avan_connect_url)

In [ ]:
avan_engine.execute('select 1').fetchall()

In [ ]:
query = f'''
select patientid, facilityid, masterpatientid
from view_ods_facility_patient
where facilityid in (select facilityid from view_ods_facility where lineofbusiness = 'SNF')
'''

master_patient_lookup = pd.read_sql(query, con=avan_engine)
master_patient_lookup['masterpatientid'] = 'avante_' + master_patient_lookup.masterpatientid.astype(str)
master_patient_lookup.to_parquet(data_path/'avante_master_patient_lookup.parquet')

In [ ]:
master_patient_lookup.head()

In [ ]:
query = f'''
select patientid, facilityid, dateoftransfer, transferreason
from view_ods_hospital_transfers_transfer_log_v2
where dateoftransfer between '{train_start_date}' and '{train_end_date}'
and facilityid in (select facilityid from view_ods_facility where lineofbusiness = 'SNF')
'''

patient_rehosps = pd.read_sql(query, con=avan_engine)
patient_rehosps = patient_rehosps.merge(master_patient_lookup, on=['patientid', 'facilityid'])
patient_rehosps.to_parquet(data_path/'avante_patient_rehosps.parquet')

In [ ]:
patient_rehosps.head()

In [ ]:
query = f'''
select masterpatientid, gender, dateofbirth, primarylanguage
from view_ods_master_patient
'''

patient_demographics = pd.read_sql(query, con=avan_engine)
patient_demographics['masterpatientid'] = 'avante_' + patient_demographics.masterpatientid.astype(str)
patient_demographics.to_parquet(data_path/'avante_patient_demographics.parquet')

In [ ]:
patient_demographics.head()

In [ ]:
query = f'''
select clientid as patientid, censusdate, facilityid, bedid
from view_ods_daily_census_v2
where censusdate between '{train_start_date}' and '{train_end_date}'
and facilityid in (select facilityid from view_ods_facility where lineofbusiness = 'SNF')
'''

patient_census = pd.read_sql(query, con=avan_engine)
patient_census = patient_census.merge(master_patient_lookup, on=['patientid','facilityid'])
patient_census.to_parquet(data_path/'avante_patient_census.parquet')

In [ ]:
patient_census.head()

In [ ]:
query = f'''
select patientid, facilityid, activitydate, principal, admission, other1, other2, other3, other4, other5, other6,
other7, other8, other9, other10, other11, other12, other13, other14, other15, other16, other17, other18, other19,
other20, other21
from view_ods_patient_diagnosis_sheet
where activitydate between '{train_start_date}' and '{train_end_date}'
and facilityid in (select facilityid from view_ods_facility where lineofbusiness = 'SNF')
'''

patient_diagnosis = pd.read_sql(query, con=avan_engine)
patient_diagnosis = patient_diagnosis.merge(master_patient_lookup, on=['patientid','facilityid'])
patient_diagnosis.to_parquet(data_path/'avante_patient_diagnosis.parquet')

In [ ]:
patient_diagnosis.head()

In [ ]:
query = f'''
select clientid as patientid, facilityid, date, vitalsdescription, value, diastolicvalue
from view_ods_Patient_weights_vitals
where date between '{train_start_date}' and '{train_end_date}'
and facilityid in (select facilityid from view_ods_facility where lineofbusiness = 'SNF')
and clientid in (select distinct clientid from view_ods_daily_census_v2 where censusdate between '{train_start_date}' and '{train_end_date}')
'''

patient_vitals = pd.read_sql(query, con=avan_engine)
patient_vitals = patient_vitals.merge(master_patient_lookup, on=['patientid', 'facilityid'])
patient_vitals.to_parquet(data_path/'avante_patient_vitals.parquet')

In [ ]:
patient_vitals.head()

In [ ]:
query = f'''
select distinct patientid, facilityid, orderdate, gpiclassdescription, gpisubclassdescription, orderdescription
from view_ods_physician_order_list_v2 a
inner join view_ods_physician_order_list_med b
on a.PhysicianOrderID = b.PhysiciansOrderID 
where orderdate between '{train_start_date}' and '{train_end_date}';
'''

patient_meds = pd.read_sql(query, con=avan_engine)
patient_meds = patient_meds.merge(master_patient_lookup, on=['patientid', 'facilityid'])
patient_meds.to_parquet(data_path/'avante_patient_meds.parquet')

In [ ]:
patient_meds.head()

In [ ]:
query = f'''
select distinct patientid, facilityid, orderdate, ordercategory, ordertype, orderdescription, diettype, diettexture, dietsupplement
from view_ods_physician_order_list_v2
where orderdate between '{train_start_date}' and '{train_end_date}'
and ordercategory in ('Diagnostic', 'Enteral - Feed', 'Dietary - Diet', 'Dietary - Supplements')
'''

patient_orders = pd.read_sql(query, con=avan_engine)
patient_orders = patient_orders.merge(master_patient_lookup, on=['patientid','facilityid'])
patient_orders.to_parquet(data_path/'avante_patient_orders.parquet')

In [ ]:
patient_orders.head()

In [ ]:
query = f'''
select patientid, facilityid, progressnotetype, createddate, sectionsequence, section, notetextorder, notetext
from view_ods_progress_note
where createddate between '{train_start_date}' and '{train_end_date}'
'''

patient_progress_notes = pd.read_sql(query, con=avan_engine)
patient_progress_notes = patient_progress_notes.merge(master_patient_lookup, on=['patientid','facilityid'])
patient_progress_notes.to_parquet(data_path/'avante_patient_progress_notes.parquet')

In [ ]:
patient_progress_notes.head()

In [ ]:
# detailed results seem to only start on 2019-03-03 and onwards - commenting out for now, revisit when we have more data

#query = f'''
#select c.patientid, a.resultdate, a.profiledescription, a.referencerange, a.result, a.abnormalityid, e.abnormalitydescription, b.reportdesciption, b.severityid, d.severitydescription from view_ods_result_lab_report_detail a
#left join view_ods_result_lab_report b on a.LabReportID = b.LabReportID
#left join view_ods_result_order_source c on b.ResultOrderSourceID = c.ResultOrderSourceID
#left join view_ods_result_lab_report_severity d on b.SeverityID = d.SeverityID
#left join view_ods_result_lab_test_abnormality e on a.AbnormalityID = e.AbnormalityID
#'''

#patient_lab_results = pd.read_sql(query, con=avan_engine)
#patient_lab_results.to_parquet(data_path/'patient_detailed_lab_results.parquet')

In [ ]:
#query = f'''
#select patientid, facilityid, createddate, stdalertid, alertdescription, a.triggereditemtype, description
#from [view_ods_cr_alert] a left join view_ods_cr_alert_triggered_item_type b
#on a.triggereditemtype = b.triggereditemtype
#where createddate between '{train_start_date}' and '{train_end_date}' and 
#((triggereditemid is not null) or (a.triggereditemtype is not null))
#'''
#
#patient_alerts = pd.read_sql(query, con=avan_engine)
#patient_alerts = patient_alerts.merge(master_patient_lookup, on=['patientid','facilityid'])
#patient_alerts.to_parquet(data_path/'patient_alerts.parquet')

# Get Greystone Data

In [ ]:
grey_connect_url = URL(
    drivername="mssql+pyodbc",
    username='saiva',
    password='Saiva27360!',
    host='172.0.95.223',
    database="pcc_replica_saiva",
    port=1433,
    query={'driver': 'ODBC Driver 17 for SQL Server'}
)

grey_engine = create_engine(grey_connect_url)

In [ ]:
grey_engine.execute('select 1').fetchall()

In [ ]:
query = f'''
select patientid, facilityid, masterpatientid
from dbo.ods_facility_patient
where facilityid in (select facilityid from dbo.ods_facility where lineofbusiness = 'SNF')
'''

master_patient_lookup = pd.read_sql(query, con=grey_engine)
master_patient_lookup['masterpatientid'] = 'greystone_' + master_patient_lookup.masterpatientid.astype(str)
master_patient_lookup.to_parquet(data_path/'grey_master_patient_lookup.parquet')

In [ ]:
master_patient_lookup.head()

In [ ]:
query = f'''
select patientid, facilityid, dateoftransfer, transferreason
from dbo.ods_hospital_transfers_transfer_log_v2
where dateoftransfer between '{train_start_date}' and '{train_end_date}'
and facilityid in (select facilityid from dbo.ods_facility where lineofbusiness = 'SNF')
'''

patient_rehosps = pd.read_sql(query, con=grey_engine)
patient_rehosps = patient_rehosps.merge(master_patient_lookup, on=['patientid', 'facilityid'])
patient_rehosps.to_parquet(data_path/'grey_patient_rehosps.parquet')

In [ ]:
patient_rehosps.head()

In [ ]:
query = f'''
select masterpatientid, gender, dateofbirth, primarylanguage
from dbo.ods_master_patient
'''

patient_demographics = pd.read_sql(query, con=grey_engine)
patient_demographics['masterpatientid'] = 'greystone_' + patient_demographics.masterpatientid.astype(str)
patient_demographics.to_parquet(data_path/'grey_patient_demographics.parquet')

In [ ]:
patient_demographics.head()

In [ ]:
query = f'''
select patientid, censusdate, facilityid, bedid
from dbo.ods_daily_census_lite
where censusdate between '{train_start_date}' and '{train_end_date}'
and facilityid in (select facilityid from dbo.ods_facility where lineofbusiness = 'SNF')
'''

patient_census = pd.read_sql(query, con=grey_engine)
patient_census = patient_census.merge(master_patient_lookup, on=['patientid','facilityid'])
patient_census.to_parquet(data_path/'grey_patient_census.parquet')

In [ ]:
patient_census.head()

In [ ]:
query = f'''
select patientid, facilityid, activitydate, principal, admission, other1, other2, other3, other4, other5, other6,
other7, other8, other9, other10, other11, other12, other13, other14, other15, other16, other17, other18, other19,
other20, other21
from dbo.ods_patient_diagnosis_sheet
where activitydate between '{train_start_date}' and '{train_end_date}'
and facilityid in (select facilityid from dbo.ods_facility where lineofbusiness = 'SNF')
'''

patient_diagnosis = pd.read_sql(query, con=grey_engine)
patient_diagnosis = patient_diagnosis.merge(master_patient_lookup, on=['patientid','facilityid'])
patient_diagnosis.to_parquet(data_path/'grey_patient_diagnosis.parquet')

In [ ]:
patient_diagnosis.head()

In [ ]:
query = f'''
select clientid as patientid, facilityid, date, vitalsdescription, value, diastolicvalue
from dbo.ods_Patient_weights_vitals
where date between '{train_start_date}' and '{train_end_date}'
and facilityid in (select facilityid from dbo.ods_facility where lineofbusiness = 'SNF')
and clientid in (select distinct patientid from dbo.ods_daily_census_lite where censusdate between '{train_start_date}' and '{train_end_date}')
'''

patient_vitals = pd.read_sql(query, con=grey_engine)
patient_vitals = patient_vitals.merge(master_patient_lookup, on=['patientid', 'facilityid'])
patient_vitals.to_parquet(data_path/'grey_patient_vitals.parquet')

In [ ]:
patient_vitals.head()

In [ ]:
query = f'''
select distinct patientid, facilityid, orderdate, gpiclassdescription, gpisubclassdescription, orderdescription
from dbo.ods_physician_order_list_v2 a
inner join dbo.ods_physician_order_list_med b
on a.PhysicianOrderID = b.PhysiciansOrderID 
where orderdate between '{train_start_date}' and '{train_end_date}';
'''

patient_meds = pd.read_sql(query, con=grey_engine)
patient_meds = patient_meds.merge(master_patient_lookup, on=['patientid', 'facilityid'])
patient_meds.to_parquet(data_path/'grey_patient_meds.parquet')

In [ ]:
patient_meds.head()

In [ ]:
query = f'''
select distinct patientid, facilityid, orderdate, ordercategory, ordertype, orderdescription, diettype, diettexture, dietsupplement
from dbo.ods_physician_order_list_v2
where orderdate between '{train_start_date}' and '{train_end_date}'
and ordercategory in ('Diagnostic', 'Enteral - Feed', 'Dietary - Diet', 'Dietary - Supplements')
'''

patient_orders = pd.read_sql(query, con=grey_engine)
patient_orders = patient_orders.merge(master_patient_lookup, on=['patientid','facilityid'])
patient_orders.to_parquet(data_path/'grey_patient_orders.parquet')

In [ ]:
patient_orders.head()

In [ ]:
query = f'''
select patientid, facilityid, progressnotetype, createddate, sectionsequence, section, notetextorder, notetext
from dbo.ods_progress_note
where createddate between '{train_start_date}' and '{train_end_date}'
'''

patient_progress_notes = pd.read_sql(query, con=grey_engine)
patient_progress_notes = patient_progress_notes.merge(master_patient_lookup, on=['patientid','facilityid'])
patient_progress_notes.to_parquet(data_path/'grey_patient_progress_notes.parquet')

In [ ]:
patient_progress_notes.head()

# Combine Raw Data

In [ ]:
patient_demographics = pd.concat(
    [pd.read_parquet(data_path/'avante_patient_demographics.parquet'), 
     pd.read_parquet(data_path/'grey_patient_demographics.parquet')], axis=0
)
patient_orders = pd.concat(
    [pd.read_parquet(data_path/'avante_patient_orders.parquet'), 
     pd.read_parquet(data_path/'grey_patient_orders.parquet')], axis=0
)
patient_vitals = pd.concat(
    [pd.read_parquet(data_path/'avante_patient_vitals.parquet'), 
     pd.read_parquet(data_path/'grey_patient_vitals.parquet')], axis=0
)
patient_rehosps = pd.concat(
    [pd.read_parquet(data_path/'avante_patient_rehosps.parquet'), 
     pd.read_parquet(data_path/'grey_patient_rehosps.parquet')], axis=0
)
patient_census = pd.concat(
    [pd.read_parquet(data_path/'avante_patient_census.parquet'), 
     pd.read_parquet(data_path/'grey_patient_census.parquet')], axis=0
)
patient_diagnosis = pd.concat(
    [pd.read_parquet(data_path/'avante_patient_diagnosis.parquet'), 
     pd.read_parquet(data_path/'grey_patient_diagnosis.parquet')], axis=0
)
patient_meds = pd.concat(
    [pd.read_parquet(data_path/'avante_patient_meds.parquet'), 
     pd.read_parquet(data_path/'grey_patient_meds.parquet')], axis=0
)

In [ ]:
def sorter(df, sort_keys=[]):
    return df.sort_values(by=sort_keys)

def deduper(df, unique_keys=[]):
    df = df.drop_duplicates(subset=unique_keys, keep='last')
    assert df.duplicated(subset=unique_keys).sum() == 0, f'''Still have dupes!'''
    
    return df

In [ ]:
patient_census = sorter(patient_census, sort_keys=['masterpatientid', 'censusdate'])
patient_vitals = sorter(patient_vitals, sort_keys=['masterpatientid', 'date'])
patient_orders = sorter(patient_orders, sort_keys=['masterpatientid', 'orderdate'])
patient_rehosps = sorter(patient_rehosps, sort_keys=['masterpatientid', 'dateoftransfer'])
patient_meds = sorter(patient_meds, sort_keys=['masterpatientid', 'orderdate'])

In [ ]:
patient_census = deduper(patient_census, unique_keys=['masterpatientid', 'censusdate'])
patient_demographics = deduper(patient_demographics, unique_keys=['masterpatientid'])
patient_vitals = deduper(patient_vitals, unique_keys=['masterpatientid', 'date', 'vitalsdescription'])
patient_orders = deduper(patient_orders, unique_keys=['masterpatientid', 'orderdate', 'orderdescription'])
patient_rehosps = deduper(patient_rehosps, unique_keys=['masterpatientid', 'dateoftransfer'])
patient_meds = deduper(patient_meds, unique_keys=['masterpatientid', 'orderdate', 'gpisubclassdescription'])

In [ ]:
base = pd.DataFrame({'censusdate': pd.date_range(start=train_start_date, end=train_end_date)})

In [ ]:
patient_rehosps['dateoftransfer'] = pd.to_datetime(patient_rehosps.dateoftransfer.dt.date)

In [ ]:
base2 = base.merge(patient_census, how='left', on=['censusdate'])
base3 = base2.merge(patient_demographics, how='left', on=['masterpatientid'])

del base2;

In [ ]:
vitals = patient_vitals.set_index(keys=['masterpatientid','facilityid','date']).drop(columns='patientid')

diastolic = vitals.pop('diastolicvalue')
diastolic = diastolic.dropna()

vitals = vitals.reset_index()
diastolic = diastolic.reset_index()

vitals['date'] = vitals.pop('date').dt.date
diastolic['date'] = diastolic.pop('date').dt.date

In [ ]:
vitals.tail()

In [ ]:
aggs=['median','std', 'max', 'min']
vitals_pivoted = vitals.loc[:,['masterpatientid', 'date', 'vitalsdescription', 'value']].pivot_table(index=['masterpatientid', 'date'], values='value', columns='vitalsdescription', aggfunc=aggs).reset_index()
diastolic_pivoted = diastolic.loc[:,['masterpatientid', 'date', 'diastolicvalue']].pivot_table(index=['masterpatientid', 'date'], values='diastolicvalue', aggfunc=aggs).reset_index()

In [ ]:
def clean_multi_columns(cols):
    new_cols = []
    
    for col in cols:
        if col[1] == '':
            new_cols.append(col[0])
        else:
            new_cols.append('_'.join(col))
            
    return new_cols

In [ ]:
vitals_pivoted.columns = clean_multi_columns(vitals_pivoted.columns)
diastolic_pivoted.columns = clean_multi_columns(diastolic_pivoted.columns)

In [ ]:
vitals_pivoted = vitals_pivoted.drop_duplicates(subset=['masterpatientid','date'], keep='last')
diastolic_pivoted = diastolic_pivoted.drop_duplicates(subset=['masterpatientid','date'], keep='last')

In [ ]:
vitals_final = vitals_pivoted.merge(diastolic_pivoted, how='left', on=['masterpatientid', 'date'])
vitals_final.columns = 'vtl_' + vitals_final.columns

In [ ]:
vitals_final.tail()

In [ ]:
base4 = base3.merge(vitals_final, how='left', left_on=['masterpatientid','censusdate'], right_on=['vtl_masterpatientid','vtl_date'])

In [ ]:
base4.tail()

In [ ]:
del vitals; del vitals_pivoted; del diastolic_pivoted; del vitals_final; del base3;

In [ ]:
lookup_ccs = pd.read_csv('/code/data/lookup_tables/ccs_dx_icd10cm_2019_1.csv')
lookup_ccs.columns = lookup_ccs.columns.str.replace("'","")
lookup_ccs = lookup_ccs.apply(lambda x: x.str.replace("'",""))

In [ ]:
diagnosis_long = patient_diagnosis.melt(id_vars=['masterpatientid','activitydate'], value_vars=['principal', 'admission', 'other1', 'other2', 'other3', 'other4',
       'other5', 'other6', 'other7', 'other8', 'other9', 'other10', 'other11',
       'other12', 'other13', 'other14', 'other15', 'other16', 'other17',
       'other18', 'other19', 'other20', 'other21'])

diagnosis_long['value'] = diagnosis_long.value.str.replace('.','')
diagnosis_long['activitydate'] = pd.to_datetime(diagnosis_long.activitydate).dt.date

In [ ]:
diagnosis_with_ccs = diagnosis_long.merge(lookup_ccs, how='inner', left_on=['value'], right_on=['ICD-10-CM CODE'])

In [ ]:
diagnosis_with_ccs['ccs_label'] = diagnosis_with_ccs['MULTI CCS LVL 1 LABEL'] + ' - ' + diagnosis_with_ccs['MULTI CCS LVL 2 LABEL']

In [ ]:
diagnosis_with_ccs['indicator'] = 1

In [ ]:
diagnosis_pivoted = diagnosis_with_ccs.loc[:,['masterpatientid', 'activitydate', 'ccs_label', 'indicator']].pivot_table(index=['masterpatientid', 'activitydate'], columns=['ccs_label'], values='indicator', fill_value=0).reset_index()
diagnosis_pivoted['activitydate'] = pd.to_datetime(diagnosis_pivoted.activitydate)
diagnosis_pivoted.columns = 'dx_' + diagnosis_pivoted.columns

In [ ]:
base5 = base4.merge(diagnosis_pivoted, how='left', left_on=['masterpatientid','censusdate'], right_on=['dx_masterpatientid','dx_activitydate'])

In [ ]:
del base4; del diagnosis_pivoted;

In [ ]:
patient_meds.loc[patient_meds.gpisubclassdescription.isna(), 'gpisubclassdescription'] = patient_meds.loc[patient_meds.gpisubclassdescription.isna(), 'gpiclassdescription']
patient_meds['orderdate'] = patient_meds.orderdate.dt.date
patient_meds['indicator'] = 1
meds_pivoted = patient_meds.loc[:,['masterpatientid', 'orderdate', 'gpisubclassdescription', 'indicator']].pivot_table(index=['masterpatientid', 'orderdate'], columns=['gpisubclassdescription'], values='indicator', fill_value=0).reset_index()

In [ ]:
meds_pivoted.columns = 'med_' + meds_pivoted.columns

In [ ]:
meds_pivoted = meds_pivoted.drop_duplicates(subset=['med_masterpatientid','med_orderdate'])

In [ ]:
meds_pivoted['med_orderdate'] = pd.to_datetime(meds_pivoted.med_orderdate)

In [ ]:
base6 = base5.merge(meds_pivoted, how='left', left_on=['masterpatientid', 'censusdate'], right_on=['med_masterpatientid', 'med_orderdate'])

In [ ]:
del base5; del meds_pivoted;

In [ ]:
diagnostic_orders = patient_orders.loc[patient_orders.ordercategory == 'Diagnostic']
diagnostic_orders['orderdate'] = diagnostic_orders.orderdate.dt.date
diagnostic_orders['count_indicator_diagnostic_orders'] = 1

In [ ]:
diagnostic_pivoted = diagnostic_orders.loc[:,['masterpatientid', 'orderdate', 'count_indicator_diagnostic_orders']].pivot_table(index=['masterpatientid', 'orderdate'], values=['count_indicator_diagnostic_orders'], aggfunc=sum).reset_index()

In [ ]:
diagnostic_pivoted['orderdate'] = pd.to_datetime(diagnostic_pivoted.orderdate)
diagnostic_pivoted.columns = 'order_' + diagnostic_pivoted.columns

In [ ]:
base7 = base6.merge(diagnostic_pivoted, how='left', left_on=['masterpatientid','censusdate'], right_on=['order_masterpatientid','order_orderdate'])

In [ ]:
del base6; del diagnostic_pivoted; del diagnostic_orders;

In [ ]:
diet_orders = patient_orders[patient_orders.ordercategory == 'Dietary - Diet']
diet_orders['orderdate'] = diet_orders.orderdate.dt.date
diet_orders['indicator'] = 1
diet_orders = diet_orders.drop_duplicates(subset=['masterpatientid', 'orderdate', 'diettype', 'diettexture'])

diet_type_pivoted = diet_orders.loc[:,['masterpatientid', 'orderdate', 'diettype', 'indicator']].pivot_table(index=['masterpatientid', 'orderdate'], columns=['diettype'], values='indicator', aggfunc=min).reset_index()
#diet_type_pivoted.columns = clean_multi_columns(diet_type_pivoted)
diet_type_pivoted.head()
diet_type_pivoted['orderdate'] = pd.to_datetime(diet_type_pivoted.orderdate)
diet_type_pivoted.columns = 'order_' + diet_type_pivoted.columns

diet_texture_pivoted = diet_orders.loc[:,['masterpatientid', 'orderdate', 'diettexture', 'indicator']].pivot_table(index=['masterpatientid', 'orderdate'], columns=['diettexture'], values='indicator', aggfunc=min).reset_index()
#diet_texture_pivoted.columns = clean_multi_columns(diet_texture_pivoted)
diet_texture_pivoted['orderdate'] = pd.to_datetime(diet_texture_pivoted.orderdate)
diet_texture_pivoted.columns = 'order_' + diet_texture_pivoted.columns

In [ ]:
base8 = base7.merge(diet_type_pivoted, how='left', left_on=['masterpatientid','censusdate'], right_on=['order_masterpatientid','order_orderdate'])
base8 = base7.merge(diet_texture_pivoted, how='left', left_on=['masterpatientid','censusdate'], right_on=['order_masterpatientid','order_orderdate'])

In [ ]:
del base7;

In [ ]:
diet_supplements = patient_orders[patient_orders.ordercategory == 'Dietary - Supplements']
diet_supplements['orderdate'] = diet_supplements.orderdate.dt.date
diet_supplements['indicator'] = 1
diet_supplements = diet_supplements.drop_duplicates(subset=['masterpatientid', 'orderdate', 'dietsupplement'])
                                                    
diet_supplements_pivoted = diet_supplements.loc[:,['masterpatientid', 'orderdate', 'dietsupplement', 'indicator']].pivot_table(index=['masterpatientid', 'orderdate'], columns='dietsupplement', values='indicator', aggfunc=min).reset_index()
diet_supplements_pivoted['orderdate'] = pd.to_datetime(diet_supplements_pivoted.orderdate)

In [ ]:
diet_supplements_counts = diet_supplements.groupby(['masterpatientid', 'facilityid', 'orderdate']).dietsupplement.count().reset_index().rename(columns={'dietsupplement':'count_indicator_dietsupplement'})
diet_supplements_counts['orderdate'] = pd.to_datetime(diet_supplements_counts.orderdate)

In [ ]:
diet_supplements_pivoted.columns = 'order_' + diet_supplements_pivoted.columns
diet_supplements_counts.columns = 'order_' + diet_supplements_counts.columns

In [ ]:
base9 = base8.merge(diet_supplements_pivoted, how='left', left_on=['masterpatientid','censusdate'], right_on=['order_masterpatientid','order_orderdate'])

In [ ]:
base9 = base9.merge(diet_supplements_counts, how='left', left_on=['masterpatientid','censusdate'], right_on=['order_masterpatientid','order_orderdate'])

In [ ]:
del base8;

In [ ]:
rehosp = patient_rehosps.merge(patient_census, on=['masterpatientid'])

In [ ]:
last_hosp = rehosp[rehosp.dateoftransfer < rehosp.censusdate]
last_hosp['count_prior_hosp'] = last_hosp.groupby(['masterpatientid', 'censusdate']).dateoftransfer.cumcount() + 1
last_hosp = last_hosp.groupby(['masterpatientid','censusdate']).tail(n=1).loc[:,['masterpatientid', 'censusdate', 'dateoftransfer', 'count_prior_hosp']].rename(columns={'dateoftransfer': 'last_hosp_date'})
last_hosp['days_since_last_hosp'] = (last_hosp.censusdate - last_hosp.last_hosp_date).dt.days

In [ ]:
next_hosp = rehosp[rehosp.dateoftransfer > rehosp.censusdate].groupby(['masterpatientid','censusdate']).head(n=1).loc[:,['masterpatientid', 'censusdate', 'dateoftransfer']].rename(columns={'dateoftransfer': 'next_hosp_date'})

In [ ]:
next_hosp['target_3_day_hosp'] = (next_hosp.next_hosp_date - next_hosp.censusdate) <= pd.to_timedelta('4 days')
next_hosp['target_7_day_hosp'] = (next_hosp.next_hosp_date - next_hosp.censusdate) <= pd.to_timedelta('8 days')

In [ ]:
last_hosp.columns = 'hosp_' + last_hosp.columns
next_hosp.columns = 'hosp_' + next_hosp.columns

In [ ]:
base10 = base9.merge(last_hosp, how='left', left_on=['masterpatientid','censusdate'], right_on=['hosp_masterpatientid', 'hosp_censusdate'])

In [ ]:
base10 = base10.merge(next_hosp, how='left', left_on=['masterpatientid','censusdate'], right_on=['hosp_masterpatientid', 'hosp_censusdate'])

In [ ]:
base10 = base10.loc[:,~base10.columns.duplicated()]

In [ ]:
processed_path = Path('/code/data/processed')
processed_path.mkdir(parents=True, exist_ok=True)

In [ ]:
base10 = base10.loc[:, base10.columns[~base10.columns.str.contains('_masterpatientid|_facilityid|vtl_date|activitydate|orderdate|createddate|_x$|_y$')].tolist()]
base10 = base10.drop_duplicates(subset=['masterpatientid', 'censusdate'], keep='last')
base10 = base10.loc[base10.masterpatientid.notna()]
base10.to_parquet(processed_path/'combined.parquet')

In [ ]:
combined = pd.read_parquet(processed_path/'combined.parquet')

In [ ]:
combined = combined.drop(columns=['hosp_last_hosp_date', 'hosp_next_hosp_date'])

In [ ]:
vtl_cols = [col for col in combined.columns if col.startswith('vtl')]
dx_cols = [col for col in combined.columns if col.startswith('dx')]
med_cols = [col for col in combined.columns if col.startswith('med')]
order_cols = [col for col in combined.columns if col.startswith('order')]
alert_cols = [col for col in combined.columns if col.startswith('alert')]
hosp_cols = [col for col in combined.columns if col.startswith('hosp')]
ignore_cols = [col for col in combined.columns if 'target' in col] + ['masterpatientid', 'patientid','censusdate', 'facilityid', 'bedid']

In [ ]:
def add_datepart(df, fldname, drop=True, time=False, errors="raise"):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    attr = ['Year','Month', 'Week', 'Day', 'Dayofweek',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[fldname + '_' + n] = getattr(fld.dt, n.lower())
    if drop: df.drop(fldname, axis=1, inplace=True)
        
def add_na_indicators(df, ignore_cols):
    missings = df.drop(columns=ignore_cols).isna()
    missings.columns = 'na_indictator_' + missings.columns
    missings_sums = missings.sum()
    
    return pd.concat([df, missings.loc[:, (missings_sums > 0)]], axis=1)

        
def proc_vitals(df, vtl_cols):
    ffilled = df.groupby('masterpatientid')[vtl_cols].fillna(method='ffill').reset_index()
    ffilled['masterpatientid'] = df.masterpatientid
    
    diff_1_day = ffilled.groupby('masterpatientid')[vtl_cols].diff()
    diff_1_day.columns = 'diff_1_day_' + diff_1_day.columns
    
    diff_7_day = ffilled.groupby('masterpatientid')[vtl_cols].diff(periods=7)
    diff_7_day.columns = 'diff_7_day_' + diff_7_day.columns
    
    rolling_avg_7_day = ffilled.groupby('masterpatientid')[vtl_cols].rolling(7, min_periods=1).mean().reset_index(0, drop=True)
    rolling_avg_7_day.columns = 'rol_avg_7_day_' + rolling_avg_7_day.columns
    
    rolling_avg_14_day = ffilled.groupby('masterpatientid')[vtl_cols].rolling(14, min_periods=1).mean().reset_index(0, drop=True)
    rolling_avg_14_day.columns = 'rol_avg_14_day_' + rolling_avg_14_day.columns
    
    rolling_std_7_day = ffilled.groupby('masterpatientid')[vtl_cols].rolling(7, min_periods=1).std().reset_index(0, drop=True)
    rolling_std_7_day.columns = 'rol_std_7_day_' + rolling_std_7_day.columns
    
    rolling_std_14_day = ffilled.groupby('masterpatientid')[vtl_cols].rolling(14, min_periods=1).std().reset_index(0, drop=True)
    rolling_std_14_day.columns = 'rol_std_14_day_' + rolling_std_14_day.columns
    
    df.loc[:,vtl_cols] = ffilled.loc[:, vtl_cols]
    
    df = pd.concat([df, diff_1_day, diff_7_day], axis=1) # diffs all indexed the same as original in the same order
    
    rollings = pd.concat([rolling_avg_7_day, rolling_avg_14_day, rolling_std_7_day, rolling_std_14_day], axis=1)
    df = df.merge(rollings, how='left', left_index=True, right_index=True) # rollings were sorted so we explictly join via index
    
    return df
    
def proc_dx_meds_alerts_orders(df, dx_cols, med_cols, order_cols):
    cols = dx_cols + med_cols + order_cols
    filled = df.groupby('masterpatientid')[cols].fillna(0).reset_index()
    filled['masterpatientid'] = df.masterpatientid
    
    cumsum_all_time = filled.groupby('masterpatientid')[cols].cumsum()
    cumsum_all_time.columns = 'cumsum_all_' + cumsum_all_time.columns
    
    cumsum_7_day = filled.groupby('masterpatientid')[cols].rolling(7, min_periods=1).sum().reset_index(0, drop=True)
    cumsum_7_day.columns = 'cumsum_7_day_' + cumsum_7_day.columns
    
    cumsum_15_day = filled.groupby('masterpatientid')[cols].rolling(15, min_periods=1).sum().reset_index(0, drop=True)
    cumsum_15_day.columns = 'cumsum_15_day_' + cumsum_15_day.columns
    
    #cumsum_30_day = filled.groupby('masterpatientid')[cols].rolling(30, min_periods=1).sum().reset_index(0, drop=True)
    #cumsum_30_day.columns = 'cumsum_30_day_' + cumsum_30_day.columns
    
    df = df.drop(columns=cols)
    df = pd.concat([df, cumsum_all_time], axis=1) # cumsum is indexed the same as original in the same order
    
    rollings = pd.concat([cumsum_7_day, cumsum_15_day], axis=1)
    df = df.merge(rollings, how='left', left_index=True, right_index=True) # rollings were sorted so we explictly join via index
    
    return df

def proc_demo(df):
    df['demo_gender'] = df.gender == 'M'
    df['demo_age_in_days'] = (df.censusdate - df.dateofbirth).dt.days
    df = pd.concat([df.drop(columns='primarylanguage'), pd.get_dummies(df.primarylanguage, prefix='demo_primarylanguage')], axis=1)
    df = pd.concat([df.drop(columns='facilityid'), pd.get_dummies(df.facilityid, prefix='demo_facility')], axis=1)
    
    return df
    
    

In [ ]:
combined = add_na_indicators(combined, ignore_cols)

In [ ]:
add_datepart(combined, 'censusdate', drop=False)

In [ ]:
add_datepart(combined, 'dateofbirth', drop=False)

In [ ]:
combined = proc_vitals(combined, vtl_cols)

In [ ]:
combined = proc_dx_meds_alerts_orders(combined, dx_cols, med_cols, order_cols)

In [ ]:
combined.to_parquet(processed_path/'after_vtl_and_cumsums.parquet')

In [ ]:
combined = pd.read_parquet(processed_path/'after_vtl_and_cumsums.parquet')

In [ ]:
combined = proc_demo(combined)

In [ ]:
combined = combined.drop_duplicates(subset=['masterpatientid', 'censusdate'])

In [ ]:
combined.to_parquet(processed_path/'final_processed.parquet')

In [ ]:
processed_path = Path('/code/data/processed')
final = pd.read_parquet(processed_path/'final_processed.parquet')

In [ ]:
drop_cols = ['bedid', 'beddescription', 'roomratetypedescription', 'payercode', 'patientid', 
             'gender', 'dateofbirth', 'citizenship', 'state', 'hosp_target_7_day_hosp']

In [ ]:
final = final.drop(columns=drop_cols)

In [ ]:
final = final.reset_index(drop=True)

In [ ]:
# manual check to make sure we're not including any columns that could leak data
with open('/code/columns.txt','w') as f:
    for col in final.columns:
        f.write(col + '\n')

In [ ]:
split_day = final.loc[:,'censusdate'].iloc[round(final.shape[0] * (1-.2))]

In [ ]:
train = final.loc[final.censusdate <= split_day]
valid = final.loc[final.censusdate > split_day]

train.to_pickle(processed_path/'train.pickle')
valid.to_pickle(processed_path/'valid.pickle')

In [ ]:
processed_path = Path('/code/data/processed')
train = pd.read_pickle(processed_path/'train.pickle')
valid = pd.read_pickle(processed_path/'valid.pickle')

In [ ]:
train_mask = train['cumsum_all_dx_Diseases of the respiratory system - Chronic obstructive pulmonary disease and bronchiectasis [127.]'] > 0
valid_mask = valid['cumsum_all_dx_Diseases of the respiratory system - Chronic obstructive pulmonary disease and bronchiectasis [127.]'] > 0

In [ ]:
print(f'{len(train[train_mask].masterpatientid.unique())} patients out of {len(train.masterpatientid.unique())} have COPD associated with them')

In [ ]:
train_patients = train[train_mask].masterpatientid.unique()
valid_patients = valid[valid_mask].masterpatientid.unique()

In [ ]:
train = train[train.masterpatientid.isin(train_patients)]
valid = valid[valid.masterpatientid.isin(valid_patients)]

In [ ]:
def fill_na_train(df):
    has_na = df.isna().sum() > 0
    d = df.loc[:, has_na].median()
    df = df.fillna(d)
    
    return df, d

def fill_na_valid(df, na_filler):
    return df.fillna(na_filler)

In [ ]:
# fill in any remaining na's - now that we're not forwardfilling past info it's not correct to use a global imputation
# hence we impute on the train and apply to the valid
train, na_filler = fill_na_train(train)
valid = fill_na_valid(valid, na_filler)

In [ ]:
train.to_pickle(processed_path/'copd_train_filled.pickle')
valid.to_pickle(processed_path/'copd_valid_filled.pickle')

In [ ]:
processed_path = Path('/code/data/processed')
train = pd.read_pickle(processed_path/'copd_train_filled.pickle')
valid = pd.read_pickle(processed_path/'copd_valid_filled.pickle')

In [ ]:
train.censusdate.min(), train.censusdate.max(), train.hosp_target_3_day_hosp.mean()

In [ ]:
valid.censusdate.min(), valid.censusdate.max(), valid.hosp_target_3_day_hosp.mean()

In [ ]:
def prep(df):
    drop_cols = ['censusdate', 'masterpatientid']
    drop_cols = drop_cols + [col for col in df.columns if 'target' in col]
    
    y = df.hosp_target_3_day_hosp.astype('float32').values
    x = df.drop(columns=drop_cols).reset_index(drop=True).astype('float32')
    idens = df.loc[:,['masterpatientid','censusdate']]
    
    return x, y, idens

In [ ]:
train_x, train_y, train_idens = prep(train)
valid_x, valid_y, valid_idens = prep(valid)

In [ ]:
import pickle;
with open('/code/data/processed/copd_train_x.pickle','wb') as f: pickle.dump(train_x, f, protocol=4)
with open('/code/data/processed/copd_train_y.pickle','wb') as f: pickle.dump(train_y, f, protocol=4)
with open('/code/data/processed/copd_train_idens.pickle','wb') as f: pickle.dump(train_idens, f, protocol=4)
with open('/code/data/processed/copd_valid_x.pickle','wb') as f: pickle.dump(valid_x, f, protocol=4)
with open('/code/data/processed/copd_valid_y.pickle','wb') as f: pickle.dump(valid_y, f, protocol=4)
with open('/code/data/processed/copd_valid_idens.pickle','wb') as f: pickle.dump(valid_idens, f, protocol=4)

In [ ]:
import pickle;
with open('/code/data/processed/copd_train_x.pickle','rb') as f: train_x = pickle.load(f)
with open('/code/data/processed/copd_train_y.pickle','rb') as f: train_y = pickle.load(f)
with open('/code/data/processed/copd_train_idens.pickle','rb') as f: train_idens = pickle.load(f)
with open('/code/data/processed/copd_valid_x.pickle','rb') as f: valid_x = pickle.load(f)
with open('/code/data/processed/copd_valid_y.pickle','rb') as f: valid_y =pickle.load(f)
with open('/code/data/processed/copd_valid_idens.pickle','rb') as f: valid_idens =pickle.load(f)

In [ ]:
param_grid = {
    'n_estimators':[500],
    'feat_select_threshold': ['16*median','32*median'],
    'max_features': ['auto'], 
    'min_samples_leaf': [100, 150], 
    'class_weight': [None],
}

In [ ]:
mlflow.set_experiment('copd_target_hosp_3_day')

In [ ]:
for config in ParameterGrid(param_grid):
    print(f'Trying hyperparamters: {config}')
    
    with mlflow.start_run():
        feat_est = forest.RandomForestClassifier(
            n_estimators=config['n_estimators'],
            max_features=config['max_features'],
            min_samples_leaf=config['min_samples_leaf'],
            class_weight=config['class_weight'],
            n_jobs=-1
        )

        feat_selector = SelectFromModel(feat_est, threshold=config['feat_select_threshold'])
        train_x_new = feat_selector.fit_transform(train_x, train_y)

        clf = forest.RandomForestClassifier(
            n_estimators=config['n_estimators'],
            max_features=config['max_features'],
            min_samples_leaf=config['min_samples_leaf'],
            class_weight=config['class_weight'],
            n_jobs=-1
        )

        clf.fit(train_x_new, train_y)

        valid_x_new = feat_selector.transform(valid_x)

        train_preds = clf.predict_proba(train_x_new)
        valid_preds = clf.predict_proba(valid_x_new)

        for param in config:
            log_param(param, config[param])

        #log_metric('train_aucroc', roc_auc_score(train_y, [pred[1] for pred in train_preds]))
        #log_metric('train_ap', average_precision_score(train_y, [pred[1] for pred in train_preds]))
        log_metric('valid_aucroc', roc_auc_score(valid_y, [pred[1] for pred in valid_preds]))
        log_metric('valid_ap', average_precision_score(valid_y, [pred[1] for pred in valid_preds]))

        log_model(feat_selector, 'feat_selector')
        log_model(clf, "model")

        feature_selected_features = pd.DataFrame(zip(train_x.columns[feat_selector.get_support()], clf.feature_importances_), columns=['feature', 'rf_importance']).sort_values('rf_importance', ascending=False)
        feature_selected_features.to_csv('./feature_selected_features.csv', index=False)
        log_artifact('./feature_selected_features.csv')

        input_features = pd.DataFrame(train_x.columns, columns=['feature'])
        input_features.to_csv('./input_features.csv', index=False)
        log_artifact('./input_features.csv')

In [ ]:
feat_est = forest.RandomForestClassifier(
    n_estimators=1000,
    max_features='auto',
    min_samples_leaf=200,
    class_weight=None,
    n_jobs=-1
)

feat_selector = SelectFromModel(feat_est, threshold='32*median')
train_x_new = feat_selector.fit_transform(train_x, train_y)

clf = forest.RandomForestClassifier(
    n_estimators=1000,
    max_features='auto',
    min_samples_leaf=200,
    class_weight=None,
    n_jobs=-1
)

clf.fit(train_x_new, train_y)

valid_x_new = feat_selector.transform(valid_x)
valid_preds = clf.predict_proba(valid_x_new)

In [ ]:
average_precision_score(valid_y, [pred[1] for pred in valid_preds])

In [ ]:
valid_x_shap = pd.concat([valid_x.loc[:, feat_selector.get_support()].reset_index(drop=True), valid_idens.reset_index(drop=True)], axis=1)
valid_x_shap['preds'] = [pred[1] for pred in valid_preds]
valid_x_shap['target'] = valid_y

In [ ]:
valid_x_shap = valid_x_shap[valid_x_shap.censusdate == pd.to_datetime('2019-02-20')]

In [ ]:
sorted_valid = valid_x_shap.sort_values(by='preds', ascending=False)
sorted_valid_dr = sorted_valid.drop(columns=['preds', 'masterpatientid', 'censusdate'])
idens = sorted_valid.loc[:, ['masterpatientid', 'censusdate', 'preds', 'target']]

In [ ]:
tt = sorted_valid_dr.head(10)

In [ ]:
explainer = shap.TreeExplainer(clf)

In [ ]:
shap_values = explainer.shap_values(tt)

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][0], tt.iloc[0])

In [ ]:
out = []

for i in range(0,10):
    shaps = pd.DataFrame({'feature_name':tt.columns, 'shap_value': shap_values[1][i], 'feature_value': tt.iloc[i]}).sort_values(by='shap_value', ascending=False)
    shaps = shaps.head(n=10)
    shaps['masterpatientid'] = hash(str(idens.iloc[i].masterpatientid))
    shaps['censusdate'] = idens.iloc[i].censusdate
    shaps['prediction'] = idens.iloc[i].preds
    shaps['rehosped'] = idens.iloc[i].target
    out.append(shaps)
    

In [ ]:
pd.concat(out).to_csv('/code/data/copd_model_2019-02-20.csv', index=False)

In [ ]:
pd.DataFrame({'feature_name':tt.columns, 'shap_value': shap_values[1][0], 'feature_value': tt.iloc[0]}).sort_values(by='shap_value', ascending=False)